# Load libraries and data

In [1]:
### Load libraries 
import os
import numpy as np
import pandas as pd

from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

from ipywidgets import *
from IPython.display import clear_output, display
from IPython.core.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

import qgrid
from traitlets import All

import helpers.research_helpers as app

from collections import OrderedDict

"""
CSV_SEPARATOR = ','     # '\t'

dict_filter_by = {}     # Used by keep_grid_show_filter

COL_COUNT = 'count'
COL_KEEP = 'keep'

COL_EMAIL = 'email'
COL_EXPIRY_DATE = 'expiration_date'
COL_EXPIRY_RANGE_CALCULATED = 'expiration_range_calculated'
COL_MEMBER_ID = 'member_id'
COL_MEMBERSHIP_ITEM = 'membership_item'

COL_EMAIL_DOMAIN = 'email_domain'
COL_INDUSTRY = 'industry'
COL_ORG_TYPE = 'organization_type'
COL_PUB_PRIVATE_CALCULATED = 'pub_private_calculated'

COL_JOB_TITLE = 'job_title'
COL_BOSS_TITLE = 'supervisor_title'
COL_NUM_OVERSEEN = 'employee_oversee'
COL_TITLE_CALCULATED = 'title_calculated'

COL_REGION_CALCULATED = 'region_calculated'
COL_LOCATION_CALCULATED = 'location_calculated'

EXPIRED_IN_MORE_THAN_180 = '(1) Expired more than 180 days'
EXPIRED_WITHIN_180 = '(2) Expired within 180 days'
EXPIRED_WITHIN_90 = '(3) Expired within 90 days'
EXPIRED_WITHIN_30 = '(4) Expired within 30 days'
EXPIRES_WITHIN_30 = '(5) Expires within 30 days'
EXPIRES_WITHIN_90 = '(6) Expires within 90 days'
EXPIRES_WITHIN_180 = '(7) Expires within 180 days'
EXPIRES_IN_MORE_THAN_180 = '(8) Expires in more than 180'

KEEP_LIST_SURVEY_DIRS = []

KEEP_LIST_EXPIRES_IN = [
    EXPIRES_WITHIN_30,
    EXPIRES_WITHIN_90,
    EXPIRES_WITHIN_180,
    EXPIRES_IN_MORE_THAN_180
    ]

KEEP_LIST_JOB_TITLES = [
    "",
    "Administrative Assistant ",
    "Administrator",
    "Asst. or Assoc. Vice Pres",
    "CHRO, CHCO",
    "Coordinator",
    "Director or Asst/Assoc Director",
    "Legal Counsel",
    "Manager, Generalist",
    "Other",
    "Partner, Principal",
    "President, CEO, Chairman",
    "Representative, Associate",
    "Specialist",
    "Supervisor",
    "VP or Asst/Assoc VP"
    ]

KEEP_LIST_MEMBERSHIP_ITEMS = [
    "Affinity Membership",
    "Corporate Membership",
    "Corporate Membership-three month extention",
    "Global Online Comp Membership - 1 Month",
    "Global Online Comp Membership - 3 Month",
    "Global Online Comp Membership - 6 Month",
    "Global Online Membership",
    "Global Online Membership - 2 Year",
    "Global Online Membership - 3 Year",
    "Professional Comp Membership - 1 Month",
    "Professional Comp Membership - 3 Month",
    "Professional Comp Membership - 6 Month",
    "Professional Membership",
    "Professional Membership - 2 Year",
    "Professional Membership - 3 Year",
    "Professional Membership - 6 Month",
    "SHRM Complimentary Membership - 1 year",
    "SHRM Life Membership",
    "SHRM Life Membership Renewal"
    ]

KEEP_LIST_REGIONS = [
    "",
    "Northeast",
    "South",
    "Midwest",
    "West"
    ]

    
def column_select_REMOVE_ME(option_array, selection = None):
    if selection:
        dd = widgets.Dropdown(
            options=option_array,
            value=selection,
            description='Please Select',
            disabled=False,
        )
    else:
        dd = widgets.Dropdown(
            options=option_array,
            description='Please Select',
            disabled=False,
        )
    
    return dd

def keep_grid_show_filter_REMOVE_ME(df, column_to_group_by, column_to_count, keep_array, bkeep_all = False):
    action_name  = column_to_group_by + '_'

    #dict_filter_by = {}
    dict_filter_by[action_name + 'label'] = widgets.HTML()
    dict_filter_by[action_name + 'label'].value = '<center><h3>Your current record count is <u>{}</u></h3></center>'.format(len(df))
    dict_filter_by[action_name + 'results_widgets'] = widgets.HBox([dict_filter_by[action_name + 'label']])

    def handle_all_events(event, qgrid_widget):
        df_testing = keep_grid_apply_filter(dict_filter_by[action_name + 'qgrid_sheet_to_keep'], column_to_group_by, df)
        dict_filter_by[qgrid_widget.action_name + 'label'].value = '<h3>Your possible record count is <b><u>{}</u></b> out of <u>{}</u></h3>'.format(len(df_testing), len(df))

    # Allow user's to select which groups, if any, they want to keep
    dict_filter_by[action_name + 'qgrid_sheet_to_keep'] = __keep_grid_show_filter(df, column_to_group_by, column_to_count, keep_array)
    setattr(dict_filter_by[action_name + 'qgrid_sheet_to_keep'], 'action_name', action_name)
    handle_all_events(None, dict_filter_by[action_name + 'qgrid_sheet_to_keep'])

    dict_filter_by[action_name + 'qgrid_sheet_to_keep'].on('cell_edited', handle_all_events)
    dict_filter_by[action_name + 'dashboard'] = widgets.VBox([dict_filter_by[action_name + 'results_widgets'], dict_filter_by[action_name + 'qgrid_sheet_to_keep']])
    display(dict_filter_by[action_name + 'dashboard'])

    return dict_filter_by[action_name + 'qgrid_sheet_to_keep']

def __keep_grid_show_filter_REMOVE_ME(df, column_to_group_by, column_to_count, keep_array, bkeep_all = False):
    msg_html = '<h1>Important:</h1></br><b>Please examine the list below and check/uncheck any record types you do not wish to use.</br></br>NOTE: </b><span>The default keep list has already been pre-selected.</span>'
    display(HTML(msg_html))
    
    if column_to_count is None:
        df_possible = df
        df_possible = df_possible.reset_index()
    else:
        df[column_to_group_by] = df[column_to_group_by].fillna('')
        df_possible = df.groupby(column_to_group_by)[column_to_count].nunique()
        df_possible = df_possible.reset_index()
        df_possible = df_possible.rename(columns={column_to_count: COL_COUNT})

    if keep_array is None:
        df_possible = df_possible.sort_values([column_to_group_by], ascending = (True))
    else:
        df_possible[COL_KEEP] = False               # set default value to False
        df_possible.astype({COL_KEEP: 'bool'}).dtypes
        # update 'keep' column based on presence of MEMBERSHIP_ITEM in keep list
        for i, row in df_possible.iterrows():
            bKeep = False
            if row[column_to_group_by] in keep_array:
                bKeep = True
            if bkeep_all:
                bKeep = True
            df_possible.at[i,COL_KEEP] = bKeep

        df_possible = df_possible.sort_values([COL_KEEP, column_to_group_by], ascending = (True, True))
        
    #sheet_keep = from_dataframe(df_possible)
    #return sheet_keep

    qgrid_widget = qgrid.show_grid(df_possible, show_toolbar=False, grid_options={"maxVisibleRows": 10})
    return qgrid_widget

def keep_grid_apply_filter_REMOVE_ME(qgrid_sheet_to_keep, col_name, df, bShowUpdate=False):
    ### Apply filtering
    ### qgrid method to convert sheet to df
    df_filter_by = qgrid_sheet_to_keep.get_changed_df()
    df_filter_by = df_filter_by[df_filter_by[COL_KEEP] == True]

    arr_keep = df_filter_by[col_name].tolist()
    
    len_before_filter = len(df)
    df_filtered_list = df[df[col_name].isin(arr_keep)] 
    len_after_filter = len(df_filtered_list)

    if bShowUpdate:
        print('The filter was successfully applied to the data frame.')
        print('There are now {} out of {} items available for use.'.format(len_after_filter, len_before_filter))

    if bShowUpdate:
        return df_filtered_list, arr_keep
    else:
        return df_filtered_list

def load_csv_REMOVE_ME(file_name):
    df = pd.read_csv (file_name , sep=CSV_SEPARATOR)
    df.columns= df.columns.str.lower()
    
    # convert column to datetime
    df[COL_EXPIRY_DATE] = pd.to_datetime(df[COL_EXPIRY_DATE])
    
    #print('There are {} email addresses available for use.'.format(len(df_all_active_roster)))
    return df
"""

app.OPTIONS_EXISTING_PROJECTS = app.load_existing_projects(app.OPTIONS_EXISTING_PROJECTS)

print('Notebook initialized.')

Notebook initialized.


In [2]:
### Load most recent, cleaned and transformed membership list from '../sampling/Member Lists/all_active.csv' 

df_complete_list = app.load_csv(os.path.join(app.FLDR_SAMPLING, app.FLDR_MEMBER_LISTS, app.FNAME_ACTIVE_ROSTER_CSV))
app.dict_dataframes['Initial list'] = df_complete_list.copy()

print(str(len(df_complete_list)) + ' records loaded from ' + os.path.join(app.FLDR_SAMPLING, app.FLDR_MEMBER_LISTS, app.FNAME_ACTIVE_ROSTER_CSV))

294201 records loaded from ../sampling/Member Lists/all_active.csv


In [3]:
### Define additional variables which are dependant on the dataset being loaded first 

app.OPTIONS_CSV_COLUMNS = np.sort(np.array(df_complete_list.columns.unique().tolist()))
app.OPTIONS_LOCATIONS = np.sort(np.array(df_complete_list[app.COL_LOCATION_CALCULATED].unique().tolist()))

# Define existing or new project - New Version

In [4]:
# Required Functions 

def on_new_existing_clicked(btn):
    print(btn)
    print('function: on_new_existing_clicked(btn)')

                              # option_array, selection = None, new_option = None, label = 'Please Select'
def build_dropdown_widgets(key, option_array, option_selected, label, on_click_callback=None, dict_filter_by=None):
    dropdown = app.build_dropdown(option_array, option_selected, label)
    go_btn = widgets.Button(value="new_existing", description="Go | {}".format(key), layout=widgets.Layout(width="500px"))
    go_btn.on_click(on_click_callback)

    toolbar_hbox = widgets.HBox(children=[dropdown, go_btn])

    if dict_filter_by is not None:
        dict_filter_by[key + '_dropdown'] = dropdown

    return toolbar_hbox

def __remove_prev_samples(df_left, base_path, folder_name, filename): 
    #print(os.path.join(base_path, folder_name, filename))
    # 1. load file into df
    df_sample = pd.read_csv (os.path.join(base_path, folder_name, filename) , sep=app.CSV_SEPARATOR, header='infer')
    # 2. convert column names to lower case
    df_sample.columns=df_sample.columns.str.lower()
    # 3. rename variations of 'email' to simply 'email'
    df_sample.rename(columns={"email_addr": "email", "email_address": "email", "emailaddr": "email"}, inplace = True)
    # 3.x drop all other colums
    df_temp = pd.DataFrame(columns=['email'])
    df_temp['email'] = df_sample['email']
    # 3.x add new column called 'prev_sampled' and set value to 1
    df_temp['prev_sample'] = 1
    # 3.x pd.merge(left=df_left, right=df_right, how='left', left_on='email', right_on='email')
    df_left = pd.merge(left=df_left, right=df_temp, how='left', left_on='email', right_on='email')
    # 3.x drop, or filter out, where 'prev_sampled' == 1
    df_left['prev_sample'] = df_left['prev_sample'].fillna(0)
    # 4 go back to step and repeat all of step 3 using member_id (IF IT EXISTS)
    df_left = df_left[df_left['prev_sample'] == 0]
    df_left = df_left.drop(columns=['prev_sample'])
    #print(str(len(df_left))) 
    return df_left

def remove_prev_samples(df_complete_list, df_previous_sample_files, dropdown):
    count_orig = len(df_complete_list)

    for index, row in df_previous_sample_files.iterrows():
        df_complete_list = __remove_prev_samples(df_complete_list, app.FLDR_SAMPLING, dropdown.value, row[COL_NEW_EXISTING_FILENAME])

    count_new = len(df_complete_list)
    print('You started with {:,} possible members.  We removed {:,}.  You have {:,} remaining.'.format(count_orig, (count_orig - count_new), count_new))
    
    return df_complete_list



#app.dict_filter_by[key + '_toolbar_vbox'] = build_dropdown_widgets(key)
#display(app.dict_filter_by[key + '_toolbar_vbox'])

In [38]:
# New or Existing Project 

KEEP_LIST_FILES = np.array([])
COL_NEW_EXISTING_FILENAME = 'filename'
COL_NEW_EXISTING_MEMBER_ID = 'member_id'

def revert_dict_state(revert_key, dict):
    dict_dataframes = {}
    dict_keys_to_remove = {}
    bFoundKey = False
    
    for key in dict:
        if bFoundKey:
            dict_keys_to_remove[key] = 1
            
        if key == revert_key:
            bFoundKey = True    

    for key in dict_keys_to_remove:
        removed_value = dict.pop(key)
        print('Removed: {}'.format(key))

    return dict


def get_dict_keys(dict):
    keys = []
    for key in dict:
        key_text = '{}'.format(key)
        key_length = '{:,}'.format(len(dict[key]))
        key_length = '{: >7}'.format(key_length)
        keys.append('{} | {}'.format(key_length, key_text))
    return keys

def on_revert_clicked(btn):
    with tab.children[4].children[1]:
        clear_output()
    
        if len(app.dict_dataframes) > 1:
            dropdown = tab.children[4].children[0].children[0]
            app.dict_dataframes = revert_dict_state(dropdown.value.split(' | ')[-1], app.dict_dataframes)
            dropdown.options = get_dict_keys(app.dict_dataframes)
            return_value = 'Finished reverting values.'
        else:
            return_value = 'Nothing to do.'
    
        print(return_value)
    
    return return_value

def grid_show_csv_listings(dropdown_value, dict_filter_by, key = None):
    KEEP_LIST_FILES = np.array([])
    #KEEP_LIST_FILES = []
    if key is None:
        key = dropdown_value
    
    #if dropdown_value != app.OPTIONS_NEW_PROJECT[-1]:
    df_new_existing = pd.DataFrame(columns=[COL_NEW_EXISTING_FILENAME, COL_NEW_EXISTING_MEMBER_ID])

    dirs = os.scandir(path = app.FLDR_SAMPLING + '/' + dropdown_value)
    ndx = 0
    for entry in dirs :
        #ndx = ndx + 1
        if entry.is_file() and entry.name[-4:] == '.csv':
            if 'forQualtrics' in entry.name:
                KEEP_LIST_FILES = np.append(KEEP_LIST_FILES, entry.name)
                #KEEP_LIST_FILES.append(entry.name)

            new_row = {COL_NEW_EXISTING_FILENAME:entry.name, COL_NEW_EXISTING_MEMBER_ID:ndx}
            df_new_existing = df_new_existing.append(new_row, ignore_index=True)

    dict_filter_by[key + '_input_df'] = df_new_existing
    dict_filter_by[key + '_keep_list_files'] = KEEP_LIST_FILES
    
    return app.keep_grid_show_filter(df_new_existing, COL_NEW_EXISTING_FILENAME, app.COL_MEMBER_ID, KEEP_LIST_FILES)

def tab_setup_new_existing():
    key = 'new_existing'
    outputs = {i: widgets.VBox() for i in range(0,3)}
    
    options_array = app.OPTIONS_EXISTING_PROJECTS.copy()
    option_array = np.insert (option_array, 0, app.OPTIONS_NEW_PROJECT[-1])
    outputs[0].children = [build_dropdown_widgets(key, option_array, app.OPTIONS_NEW_PROJECT[-1], on_new_existing_clicked, app.dict_filter_by), widgets.Output()]
    outputs[1].children = [widgets.VBox(), widgets.Output()]
    outputs[2].children = [widgets.VBox(), widgets.Output()]

    # add the Output widgets as tab childen
    tab_strip = widgets.Tab()
    tab_strip.children = list(outputs.values())

    for i, title in outputs.items():
        if i == 0:
            tab_strip.set_title(i, 'New or Existing Proj.')
        if i == 1:
            tab_strip.set_title(i, 'Filter by...')
        if i == 2:
            tab_strip.set_title(i, 'Apply Changes')
    
    return tab_strip

def tab_select_new_existing(widget):
    key = 'new_existing'
    tab_idx_prev = widget['old']
    tab_idx = widget['new']
    
    #if tab_idx == 0:
    #    output_widget = tab.children[tab_idx].children[-1]
    #else:
    #    #print(type(tab.children[tab_idx]))
    #    output_widget = tab.children[tab_idx].children[-1]
    
    #print('-{}-'.format(type(tab.children[tab_idx])))
    #print('-{}-'.format(type(tab.children[tab_idx].children[0])))
    #print('-{}-'.format(type(tab.children[tab_idx].children[0].children[0])))
    #print('-{}-'.format(type()))
    
    output_widget = tab.children[tab_idx].children[-1]
    with output_widget:
        if tab_idx != 0:
            dropdown_criteria = tab.children[0].children[0].children[0]
            #print(dropdown_criteria.value)
            #dropdown = tab.children[0].children[0].children[0]
        
        if tab_idx == 1:
            clear_output()
            
            #dropdown_criteria = tab.children[0].children[0].children[0]
            
            if dropdown_criteria.value == 'Remove previous samples':
                option_array = app.OPTIONS_EXISTING_PROJECTS
                #option_array = app.OPTIONS_EXISTING_PROJECTS.copy()
                #option_array = np.insert (option_array, 0, app.OPTIONS_NEW_PROJECT[-1])
            else:
                option_array = app.OPTIONS_CSV_COLUMNS
            
            
            dropdown = tab.children[1].children[0].children[0].options = option_array
            
            #display(app.build_dropdown(option_array, None, ''))
            
            #tab.children[0].children[0].children[0].options = ['A', 'B']
            #print('-{}-'.format(len(tab.children[tab_idx].children[0].children)))
            #print('-{}-'.format(type(tab.children[tab_idx].children[0])))
            #print('-{}-'.format(type(tab.children[tab_idx].children[1])))
            #print('-{}-'.format(len(tab.children[tab_idx].children[0].children[0].value)))
            
            #print('-{}-'.format(type(tab.children[tab_idx])))
            #print('-{}-'.format(type(tab.children[tab_idx].children[2])))
            #print('-{}-'.format(type(tab.children[tab_idx].children[0].children[0])))
            
        if tab_idx == 2:
            clear_output()
            
            #print('running long function')
            #display(tab.children[0].children[0].value)
            dropdown = tab.children[1].children[0].children[0]
            if dropdown.value != app.OPTIONS_NEW_PROJECT[-1] and dropdown.value != '' and dropdown.value is not None:
                pass
                #app.dict_filter_by[key + '_qgrid_sheet_to_keep'] = grid_show_csv_listings(dropdown.value, app.dict_filter_by, key)
                
                #new_existing_vbox.children += (new_existing_important_msg_hbox,)
                #grid_new_existing = grid_show_csv_listings(dropdown.value)
                #grid_show_csv_listings(dropdown.value)
                
            if dropdown_criteria.value == 'Remove previous samples':
                #grid_show_csv_listings(dropdown.value, app.dict_filter_by, key)

                KEEP_LIST_FILES = np.array([])
                df_new_existing = pd.DataFrame(columns=[COL_NEW_EXISTING_FILENAME, COL_NEW_EXISTING_MEMBER_ID])
                dirs = os.scandir(path = app.FLDR_SAMPLING + '/' + dropdown.value)
                ndx = 0
                for entry in dirs :
                    #ndx = ndx + 1
                    if entry.is_file() and entry.name[-4:] == '.csv':
                        if 'forQualtrics' in entry.name:
                            KEEP_LIST_FILES = np.append(KEEP_LIST_FILES, entry.name)
                            #KEEP_LIST_FILES.append(entry.name)

                        new_row = {COL_NEW_EXISTING_FILENAME:entry.name, COL_NEW_EXISTING_MEMBER_ID:ndx}
                        df_new_existing = df_new_existing.append(new_row, ignore_index=True)

                app.dict_filter_by['remove_samples_input_df'] = df_new_existing
                app.grid_work_do = app.keep_grid_show_filter(df_new_existing, COL_NEW_EXISTING_FILENAME, app.COL_MEMBER_ID, KEEP_LIST_FILES)
                #app.keep_grid_show_filter(df_new_existing, COL_NEW_EXISTING_FILENAME, app.COL_MEMBER_ID, KEEP_LIST_FILES)
                #tab.children[2].children[0].children += (app.keep_grid_show_filter(df_new_existing, COL_NEW_EXISTING_FILENAME, app.COL_MEMBER_ID, KEEP_LIST_FILES),)

            else:
                #print(type(app.dict_dataframes[list(app.dict_dataframes)[-1]]))
                #print(app.dict_dataframes[list(app.dict_dataframes)[-1]])
                #print(app.dict_dataframes[list(app.dict_dataframes)[-1]].columns)
                #                                                (df, column_to_group_by, column_to_count, keep_array, bkeep_all = False)
                #qgrid_filter_by_item = 
                app.grid_work_do = app.keep_grid_show_filter(app.dict_dataframes[list(app.dict_dataframes)[-1]], dropdown.value, app.COL_MEMBER_ID, app.get_keep_list(dropdown.value))
            
            #clear_output()
            #print(value)
            
        if tab_idx == 3:
            clear_output()
            
            import types

            def dump_obj(obj, level=0):
                for key, value in obj.__dict__.items():
                    if not isinstance(value, type):
                         print(" " * level + "%s -> %s" % (key, value))
                    else:
                        dump_obj(value, level + 2)
            
            #dump_obj(tab.children[2].children[0])
            
            #print('-{}-'.format(len(tab.children[2].children)))
            #print('-{}-'.format(type(tab.children[2].children[0].children[0])))
            #print('-{}-'.format(len(tab.children[2].children[0].children)))
            #print('-{}-'.format(len(tab.children[2].children[1].children)))
            #print('-{}-'.format(type(tab.children[2].children[0])))
            #print('-{}-'.format(type(tab.children[2].children[1])))
            #return
            
            dropdown = tab.children[1].children[0].children[0]
            if dropdown_criteria.value == 'Remove previous samples':
                #print(type(app.dict_filter_by[key + '_qgrid_sheet_to_keep']))
                #print(app.dict_filter_by.keys())
                
                #app.dict_filter_by[key + '_input_df']
                #app.dict_filter_by[key + '_keep_list_files']
    
                df_previous_sample_files, app.dict_keep_lists[dropdown.value] = app.keep_grid_apply_filter(app.grid_work_do, COL_NEW_EXISTING_FILENAME, app.dict_filter_by['remove_samples_input_df'], True)
                print('Please wait while we remove the previous samples from the current dataset...')
                df_filtered_list = remove_prev_samples(app.dict_dataframes[list(app.dict_dataframes)[-1]], df_previous_sample_files, dropdown)
                #df_filtered_list = remove_prev_samples(app.dict_dataframes['Initial list'], df_previous_sample_files, dropdown)
                app.dict_dataframes[dropdown_criteria.value + ' from ' + dropdown.value] = df_filtered_list.copy()
            
            
                #df_filtered_list, app.dict_keep_lists[dd_filter_by_column_item.value] = app.keep_grid_apply_filter(qgrid_filter_by_item, dd_filter_by_column_item.value, app.dict_dataframes[list(app.dict_dataframes)[-1]], True)
                #app.dict_dataframes['Filtered by ' + dd_filter_by_column_item.value] = df_filtered_list.copy()

            
                #df_previous_sample_files, KEEP_LIST_FILES = app.keep_grid_apply_filter(qgrid_new_existing, COL_NEW_EXISTING_FILENAME, df_new_existing, True)
                #df_complete_list = remove_prev_samples(df_complete_list, df_previous_sample_files, new_existing_dropdown)
                #app.dict_dataframes['Removed samples from current project'] = df_complete_list.copy()
            
                #print(str(len(app.dict_dataframes['Removed samples from current project'])))
                #display('sdfsdf')
            else:
                df_filtered_list, app.dict_keep_lists[dropdown.value] = app.keep_grid_apply_filter(app.grid_work_do, dropdown.value, app.dict_dataframes[list(app.dict_dataframes)[-1]], True)
                app.dict_dataframes[dropdown_criteria.value + ' - ' + dropdown.value] = df_filtered_list.copy()
                
        if tab_idx == 4:
            clear_output()
            
            tab.children[4].children[0].children[0].options = get_dict_keys(app.dict_dataframes)
            #dropdown = tab.children[4].children[0].children[0]
            #if len(app.dict_dataframes) > 1:
            #    pass
            #else:
            #    pass
            #    #print('Nothing to revert to.')
            

#tab_strip_new_existing = tab_setup_new_existing()

key = 'new_existing'
outputs = {i: widgets.VBox() for i in range(0,5)}
option_array = app.OPTIONS_EXISTING_PROJECTS.copy()
#option_array = np.insert (option_array, 0, app.OPTIONS_NEW_PROJECT[-1])

#def dropdown_criteria_eventhandler(change):
#    print(change.new)
#
#dropdown_criteria.observe(dropdown_criteria_eventhandler, names='value')

dropdown_criteria = app.build_dropdown(['Remove previous samples', 'Filter on categorical variables'], None, 'I want to...')
outputs[0].children = [widgets.VBox(children=[dropdown_criteria]), widgets.Output()]

#outputs[1].children = [build_dropdown_widgets(key, [], '', '', on_new_existing_clicked, app.dict_filter_by), widgets.Output()]
outputs[1].children = [widgets.VBox(children=[app.build_dropdown(app.OPTIONS_EXISTING_PROJECTS, None, '')]), widgets.Output()]
#outputs[0].children = [build_dropdown_widgets(key, option_array, app.OPTIONS_NEW_PROJECT[-1], '', on_new_existing_clicked, app.dict_filter_by), widgets.Output()]
outputs[2].children = [widgets.VBox(), widgets.Output()]
outputs[3].children = [widgets.VBox(), widgets.Output()]

revert_btn = widgets.Button(value="revert", description="Revert to Previous State", layout=widgets.Layout(width="500px"))
revert_btn.on_click(on_revert_clicked)
outputs[4].children = [widgets.HBox(children=[app.build_dropdown([], None, ''), revert_btn]), widgets.Output()]

# add the Output widgets as tab childen
tab = widgets.Tab()
tab.children = list(outputs.values())

for i, title in outputs.items():
    if i == 0:
        tab.set_title(i, 'Step 0 - Criteria')
    if i == 1:
        tab.set_title(i, 'Step 1 - Select')
    if i == 2:
        tab.set_title(i, 'Step 2 - Filter')
    if i == 3:
        tab.set_title(i, 'Step 3 - Apply')
    if i == 4:
        tab.set_title(i, 'View / Undo')

tab.observe(tab_select_new_existing, names='selected_index')
display(tab)

# Define existing or new project

In [ ]:
# New or Existing Project 

new_existing_dropdown = app.column_select(app.OPTIONS_EXISTING_PROJECTS, app.OPTIONS_NEW_PROJECT[-1], app.OPTIONS_NEW_PROJECT)
display(new_existing_dropdown)

In [ ]:
### If NOT <New Project> allow user to select previous samples that should be removed from the master roster list. 

KEEP_LIST_FILES = np.array([])
COL_NEW_EXISTING_FILENAME = 'filename'
COL_NEW_EXISTING_MEMBER_ID = 'member_id'

if new_existing_dropdown.value != app.OPTIONS_NEW_PROJECT[-1]:
    df_new_existing = pd.DataFrame(columns=[COL_NEW_EXISTING_FILENAME, COL_NEW_EXISTING_MEMBER_ID])
    
    dirs = os.scandir(path = app.FLDR_SAMPLING + '/' + new_existing_dropdown.value)
    ndx = 0
    for entry in dirs :
        #ndx = ndx + 1
        if entry.is_file() and entry.name[-4:] == '.csv':
            if 'forQualtrics' in entry.name:
                KEEP_LIST_FILES = np.append(KEEP_LIST_FILES, entry.name)
            
            new_row = {COL_NEW_EXISTING_FILENAME:entry.name, COL_NEW_EXISTING_MEMBER_ID:ndx}
            df_new_existing = df_new_existing.append(new_row, ignore_index=True)
    
    qgrid_new_existing = app.keep_grid_show_filter(df_new_existing, COL_NEW_EXISTING_FILENAME, app.COL_MEMBER_ID, KEEP_LIST_FILES)

## If NOT a &lt;New Project&gt; then apply filtering 

In [ ]:
### If NOT a <New Project> then apply filtering 
#
#To just get the dataframe returned by qgrid_sheet_to_keep, just run the next 2 lines s
#df_filter_by = qgrid_sheet_to_keep.get_changed_df()
#df_filter_by.head(40)


def __remove_prev_samples(df_left, base_path, folder_name, filename): 
    #print(os.path.join(base_path, folder_name, filename))
    # 1. load file into df
    df_sample = pd.read_csv (os.path.join(base_path, folder_name, filename) , sep=app.CSV_SEPARATOR, header='infer')
    # 2. convert column names to lower case
    df_sample.columns=df_sample.columns.str.lower()
    # 3. rename variations of 'email' to simply 'email'
    df_sample.rename(columns={"email_addr": "email", "email_address": "email", "emailaddr": "email"}, inplace = True)
    # 3.x drop all other colums
    df_temp = pd.DataFrame(columns=['email'])
    df_temp['email'] = df_sample['email']
    # 3.x add new column called 'prev_sampled' and set value to 1
    df_temp['prev_sample'] = 1
    # 3.x pd.merge(left=df_left, right=df_right, how='left', left_on='email', right_on='email')
    df_left = pd.merge(left=df_left, right=df_temp, how='left', left_on='email', right_on='email')
    # 3.x drop, or filter out, where 'prev_sampled' == 1
    df_left['prev_sample'] = df_left['prev_sample'].fillna(0)
    # 4 go back to step and repeat all of step 3 using member_id (IF IT EXISTS)
    df_left = df_left[df_left['prev_sample'] == 0]
    df_left = df_left.drop(columns=['prev_sample'])
    #print(str(len(df_left))) 
    return df_left

def remove_prev_samples(df_complete_list, df_previous_sample_files, dropdown):
    count_orig = len(df_complete_list)

    for index, row in df_previous_sample_files.iterrows():
        df_complete_list = __remove_prev_samples(df_complete_list, app.FLDR_SAMPLING, dropdown.value, row[COL_NEW_EXISTING_FILENAME])

    count_new = len(df_complete_list)
    print('You started with {:,} possible members.  We removed {:,}.  You have {:,} remaining.'.format(count_orig, (count_orig - count_new), count_new))
    
    return df_complete_list
    

if new_existing_dropdown.value != app.OPTIONS_NEW_PROJECT[-1]:
    df_previous_sample_files, KEEP_LIST_FILES = app.keep_grid_apply_filter(qgrid_new_existing, COL_NEW_EXISTING_FILENAME, df_new_existing, True)
    df_complete_list = remove_prev_samples(df_complete_list, df_previous_sample_files, new_existing_dropdown)
    app.dict_dataframes['Removed samples from current project'] = df_complete_list.copy()

# Undo changes, and revert to previous state

In [16]:
### Please select from the list below if you want to undo a series of filters 

def get_dict_keys(dict):
    keys = []
    for key in dict:
        key_text = '{}'.format(key)
        key_length = '{:,}'.format(len(dict[key]))
        key_length = '{: >7}'.format(key_length)
        keys.append('{} | {}'.format(key_length, key_text))
    return keys
    
#for key in app.dict_dataframes:
#    print('{} has {:,} records.'.format(key, len(app.dict_dataframes[key])))

if len(app.dict_dataframes) > 1:
    for key in app.dict_dataframes:
        print('{} has {:,} records.'.format(key, len(app.dict_dataframes[key])))
    dd_previous = app.column_select(get_dict_keys(app.dict_dataframes), app.OPTIONS_PREV_DATAFRAMES[-1], app.OPTIONS_PREV_DATAFRAMES)
    display(dd_previous)
else:
    print('Nothing to undo.')

Initial list has 294,201 records.
Remove previous samples from 2021 Benchmarking Wave 2 has 258,732 records.


AttributeError: module 'helpers.research_helpers' has no attribute 'column_select'

In [ ]:
### If needed, undo a series of filters) 

def revert_dict_state(revert_key, dict):
    dict_dataframes = {}
    dict_keys_to_remove = {}
    bFoundKey = False
    
    for key in dict:
        if bFoundKey:
            dict_keys_to_remove[key] = 1
            
        if key == revert_key:
            bFoundKey = True    

    for key in dict_keys_to_remove:
        removed_value = dict.pop(key)
        print('Removed: {}'.format(key))

    return dict

if len(app.dict_dataframes) > 1:
    if dd_previous.value != app.OPTIONS_PREV_DATAFRAMES[-1]:
        app.dict_dataframes = revert_dict_state(dd_previous.value.split(' | ')[-1], app.dict_dataframes)
else:
    print('Nothing to do.')

# Filter by... 

In [ ]:
### Show drop down with "membership_item" pre-selected 
## NOTE:  Pay attention to the variable "dd_membership_item".
## This variable is referenced twice in this cell and once in the cell directly following this cell.
########################################### 

dd_filter_by_column_item = app.column_select(app.OPTIONS_CSV_COLUMNS, app.COL_MEMBERSHIP_ITEM)

display(dd_filter_by_column_item)

In [ ]:
qgrid_filter_by_item = app.keep_grid_show_filter(app.dict_dataframes[list(app.dict_dataframes)[-1]], dd_filter_by_column_item.value, app.COL_MEMBER_ID, app.get_keep_list(dd_filter_by_column_item.value))

## Apply Changes

In [ ]:
### Apply filtering 
#
#To just get the dataframe returned by qgrid_sheet_to_keep, just run the next 2 lines s
#df_filter_by = qgrid_sheet_to_keep.get_changed_df()
#df_filter_by.head(40)

df_filtered_list, app.dict_keep_lists[dd_filter_by_column_item.value] = app.keep_grid_apply_filter(qgrid_filter_by_item, dd_filter_by_column_item.value, app.dict_dataframes[list(app.dict_dataframes)[-1]], True)

app.dict_dataframes['Filtered by ' + dd_filter_by_column_item.value] = df_filtered_list.copy()

print('\n')
print('Repeat step ("Filter by...") as many times as needed.')
print('If you need to go back to a previous state, go to Step 3')

# Filter by... membership expiration date range

In [ ]:
### Please select those expiration groups you would like to keep. 
## The default list (keep_records.KEEP_LIST_EXPIRES_IN) is already selected.

# Allow user's to select which expiration groups, if any, they want to exclude
qgrid_exp_date_range = app.keep_grid_show_filter(df_filtered_membership_item, app.COL_EXPIRY_RANGE_CALCULATED, app.COL_MEMBER_ID, app.KEEP_LIST_EXPIRES_IN)
#qgrid_sheet_to_keep

## Apply changes

In [ ]:
### Apply filtering 
#
# To just get the dataframe returned by qgrid_sheet_to_keep, just run the next 2 lines s
#df_filter_by = qgrid_sheet_to_keep.get_changed_df()
#df_filter_by.head(40)

df_filtered_exp_date_range, app.KEEP_LIST_EXPIRES_IN = app.keep_grid_apply_filter(qgrid_exp_date_range, app.COL_EXPIRY_RANGE_CALCULATED, df_filtered_membership_item, True)

# Filter by... job_title

In [ ]:
### Please select those job titles you would like to keep. 

qgrid_job_title = app.keep_grid_show_filter(df_filtered_exp_date_range, app.COL_JOB_TITLE, app.COL_MEMBER_ID, app.KEEP_LIST_JOB_TITLES)

## Apply changes

In [ ]:
### Apply filtering 

# To just get the dataframe returned by qgrid_sheet_to_keep, just run the next 2 lines s
#df_filter_by = qgrid_sheet_to_keep.get_changed_df()
#df_filter_by.head(40)

df_filtered_job_title, app.KEEP_LIST_JOB_TITLES = app.keep_grid_apply_filter(qgrid_job_title, app.COL_JOB_TITLE, df_filtered_exp_date_range, True)

# Filter by... region 

In [ ]:
### Please select those regions you would like to keep. 

qgrid_region = app.keep_grid_show_filter(df_filtered_job_title, app.COL_REGION_CALCULATED, app.COL_MEMBER_ID, app.KEEP_LIST_REGIONS)

## Apply changes 

In [ ]:
### Apply filtering 

df_filtered_region, app.KEEP_LIST_REGION = app.keep_grid_apply_filter(qgrid_region, app.COL_REGION_CALCULATED, df_filtered_job_title, True)

# Filter by... survey recency

## Apply changes

# Save output

## Get project/directory name

In [ ]:
### What is the name of this project? 

project_name_lbl = widgets.Label('Project Name: ')
project_name_txt = widgets.Text()
display(project_name_lbl)
display(project_name_txt)

In [ ]:
### Your data will be saved to... 
project_name_results_lbl = widgets.Label()
project_name_results_lbl.value = 'Your data will be saved to ' + app.FLDR_SAMPLING + '/' + project_name_txt.value + '/'
display(project_name_results_lbl)

In [ ]:
### Create folder in ../sampling/<project_name> 

new_abs_path = os.path.join(os.getcwd(), app.FLDR_SAMPLING, project_name_txt.value)

if os.path.exists(new_abs_path):
    print('The project name already exists.')
    print('Either create a new project name by rerunning the "Get project/directory name" cells, or...')
    print('Use the project name as is.  \n\nWarning - this may cause issues later!')
else:
    print('Created project folder in: \n{}'.format(os.path.join(os.getcwd(), app.FLDR_SAMPLING, project_name_txt.value)))
    os.mkdir(new_abs_path)

## Determine Sample Size

In [ ]:
seed_value = 4500
resp_rate = .03
target_complete = 1000
#sampSize <- round(if((targetComplete/respRate) < nrow(pool)) targetComplete/respRate else nrow(pool),0)
sample_size = round(target_complete / resp_rate if (target_complete / resp_rate) < len(df_filtered_list) else len(df_filtered_list))
print(sample_size)
#df_sample = df_filtered_list.sample(n = sample_size, replace = False, random_state = seed_value)

In [ ]:
## Save project files 

In [ ]:
### Save all project files

def __get_date(): 
    now = datetime.now()
    return now.strftime("%Y-%m-%d")

date_string = __get_date()
new_abs_path = os.path.join(os.getcwd(), app.FLDR_SAMPLING, project_name_txt.value)

df_complete_list.to_csv(new_abs_path + '/all_active_{}.csv'.format(date_string), header=True, index=False)
df_filtered_list.to_csv(new_abs_path + '/filtered_{}.csv'.format(date_string), header=True, index=False)
df_sample.to_csv(new_abs_path + '/sample_{}.csv'.format(date_string), header=True, index=False)

df_remaining = df_filtered_list.loc[~df_filtered_list[app.COL_MEMBER_ID].isin(df_sample[app.COL_MEMBER_ID])].copy()

df_qualtrics = df_sample[app.COL_MEMBER_ID, first_name', 'last_name', app.COL_EMAIL]
df_qualtrics.to_csv(new_abs_path + '/full_sample_N{}_forQualtrics_complete_{}.csv'.format(sample_size, date_string), header=True, index=False)

time_zone = "EST"
df_tz = df_sample[df_sample['time_zone'] == time_zone]
df_qualtrics = df_tz[app.COL_MEMBER_ID, first_name', 'last_name', app.COL_EMAIL] 
df_qualtrics.to_csv(new_abs_path + '/sample_{}_N{}_forQualtrics_{}.csv'.format(time_zone, sample_size, date_string), header=True, index=False)

time_zone = "CST"
df_tz = df_sample[df_sample['time_zone'] == time_zone]
df_qualtrics = df_tz[app.COL_MEMBER_ID, first_name', 'last_name', app.COL_EMAIL] 
df_qualtrics.to_csv(new_abs_path + '/sample_{}_N{}_forQualtrics_{}.csv'.format(time_zone, sample_size, date_string), header=True, index=False)

time_zone = "MST"
df_tz = df_sample[df_sample['time_zone'] == time_zone]
df_qualtrics = df_tz[app.COL_MEMBER_ID, first_name', 'last_name', app.COL_EMAIL] 
df_qualtrics.to_csv(new_abs_path + '/sample_{}_N{}_forQualtrics_{}.csv'.format(time_zone, sample_size, date_string), header=True, index=False)

time_zone = "PST"
df_tz = df_sample[df_sample['time_zone'] == time_zone]
df_qualtrics = df_tz[app.COL_MEMBER_ID, first_name', 'last_name', app.COL_EMAIL] 
df_qualtrics.to_csv(new_abs_path + '/sample_{}_N{}_forQualtrics_{}.csv'.format(time_zone, sample_size, date_string), header=True, index=False)
